## Load WRI Power Plant data from 2017 dataset (see https://github.com/wri/global-power-plant-database) for original source

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have a local copy in an s3 bucket here: s3://ocp-odh-os-demo-s3/data/WRI-upload/wri-pp.tsv
### To tidy the data we factor into two tables:

* **wri_plants** (all the fixed data about each plant)
* **wri_annual_gwh** (per plant/per year annual generation in GWh)

### The next step is to annotate **wri_annual_gwh** with OS-C Factor metadata

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

### Load python modules if starting from a minimal notebook

In [1]:
# %pip install python-dotenv
# %pip install boto3
# %pip install pandas

### Load Credentials

In [2]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

### Load WRI data file using pandas *read_csv*

In [3]:
import boto3
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_ENDPOINT'],
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
)

obj = s3.get_object(
    Bucket=os.environ['S3_BUCKET'],
    Key='WRI/global_power_plant_database_v_1_3/global_power_plant_database.csv'
)

In [4]:
import pandas as pd

wri_file = pd.read_csv(obj['Body'])

display(wri_file.columns)

/opt/app-root/lib64/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018',
       'generation_gwh_2019', 'generation_data_source',
       'estimated_generation_gwh_2013', 'estimated_generation_gwh_2014',
       'estimated_generation_gwh_2015', 'estimated_generation_gwh_2016',
       'estimated_generation_gwh_2017', 'estimated_generation_note_2013',
       'estimated_generation_note_2014', 'estimated_generation_note_2015',
       'estimated_generation_note_2016', 'estimated_generation_note_2017'],
      dtype='object')

### Melt the generation data into a more tidy format, dropping NA values

In [12]:
wri_plants = wri_file[['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data', 'generation_data_source']]

wri_id_vars = ['gppd_idnr']
wri_value_vars = ['generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018', 'generation_gwh_2019']
wri_annual_gwh = wri_file.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='generation_gwh')
wri_annual_gwh['year'] = wri_annual_gwh['year'].apply(lambda x: int(x.split('_')[-1]))
wri_annual_gwh.dropna(subset=['generation_gwh'],inplace=True)

display(wri_annual_gwh)

,gppd_idnr,year,generation_gwh
0,GEODB0040538,2013,NaN
1,WKS0070144,2013,NaN
2,WKS0071196,2013,NaN
3,GEODB0040541,2013,NaN
4,GEODB0040534,2013,NaN
...,...,...,...
244547,WRI1022386,2019,NaN
244548,WRI1022384,2019,NaN
244549,WRI1022380,2019,NaN
244550,GEODB0040404,2019,NaN


In [14]:
wri_id_vars = ['gppd_idnr']
wri_value_vars = ['estimated_generation_gwh_2013', 'estimated_generation_gwh_2014', 'estimated_generation_gwh_2015', 'estimated_generation_gwh_2016', 'estimated_generation_gwh_2017']
wri_estimated_gwh = wri_file.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='estimated_generation_gwh')
wri_estimated_gwh['year'] = wri_estimated_gwh['year'].apply(lambda x: int(x.split('_')[-1]))

wri_value_vars = ['estimated_generation_note_2013', 'estimated_generation_note_2014', 'estimated_generation_note_2015', 'estimated_generation_note_2016', 'estimated_generation_note_2017']
wri_estimated_note = wri_file.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='estimated_generation_note')
wri_estimated_note['year'] = wri_estimated_note['year'].apply(lambda x: int(x.split('_')[-1]))

wri_estimated_gwh = wri_estimated_gwh.merge(wri_estimated_note, on=['gppd_idnr', 'year'], validate="one_to_one")

wri_estimated_gwh.dropna(subset=['estimated_generation_gwh'],inplace=True)

display(wri_estimated_gwh)

,gppd_idnr,year,estimated_generation_gwh,estimated_generation_note
0,GEODB0040538,2013,123.77,HYDRO-V1
1,WKS0070144,2013,18.43,SOLAR-V1-NO-AGE
2,WKS0071196,2013,18.64,SOLAR-V1-NO-AGE
3,GEODB0040541,2013,225.06,HYDRO-V1
4,GEODB0040534,2013,406.16,HYDRO-V1
...,...,...,...,...
174675,WRI1022386,2017,183.79,CAPACITY-FACTOR-V1
174676,WRI1022384,2017,73.51,CAPACITY-FACTOR-V1
174677,WRI1022380,2017,578.32,HYDRO-V1
174678,GEODB0040404,2017,2785.10,CAPACITY-FACTOR-V1


In [11]:
display(wri_plants)

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_data_source
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.3220,65.1190,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009793,2017.0,NaN
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.6700,65.7950,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.6230,65.7920,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.5560,69.4787,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009795,2017.0,NaN
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.6410,69.7170,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009797,2017.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,WRI1022386,50.0,-12.9667,28.6333,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1089529,NaN,NaN
34932,ZMB,Zambia,Nkana,WRI1022384,20.0,-12.8167,28.2000,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1043097,NaN,NaN
34933,ZMB,Zambia,Victoria Falls,WRI1022380,108.0,-17.9167,25.8500,Hydro,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1033763,NaN,NaN
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,GEODB0040404,920.0,-18.3835,26.4700,Coal,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1033856,2017.0,NaN
